In [ ]:
from google.colab import drive
drive.mount('/content/drive/',force_remount=True)

Mounted at /content/drive/


In [2]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths

from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


In [8]:
dataset = r'drive/My Drive/dataset'
imagePaths = list(paths.list_images(dataset))

In [ ]:
data = []
labels = []

for imagePath in imagePaths:
  image = load_img(imagePath,target_size=(224,224))
  image = img_to_array(image)
  image = preprocess_input(image)
  data.append(image)

  label = imagePath.split(os.path.sep)[-2]
  labels.append(label)

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:932: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


In [ ]:
data = np.array(data,dtype="float32")
labels = np.array(labels)

In [ ]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
trainX,testX,trainY,testY = train_test_split(data,labels,test_size=0.20,random_state=10,stratify=labels)

In [ ]:
trainX.shape

(2456, 224, 224, 3)

In [ ]:
testX.shape

(614, 224, 224, 3)

In [ ]:
aug = ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15,horizontal_flip=True,vertical_flip=True,fill_mode='nearest') 


In [ ]:
aug

In [ ]:
baseModel = MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))

In [ ]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7,7))(headModel)
headModel = Flatten(name="Flatten")(headModel)
headModel = Dense(128,activation='relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2,activation="softmax")(headModel)

model = Model(inputs=baseModel.input,outputs=headModel)

In [ ]:
for layer in baseModel.layers:
  layer.trainable = False

In [ ]:
model.summary()

Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
______________________________________________________________________________________

In [ ]:
learning_rate = 0.0001
Epochs = 20
Batch_size = 32

opt = Adam(lr=learning_rate,decay=learning_rate/Epochs)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

model.fit(
    aug.flow(trainX,trainY,batch_size=Batch_size),
    steps_per_epoch=len(trainX)//Batch_size,
    validation_data = (testX,testY),
    validation_steps = len(testX)//Batch_size,
    epochs=Epochs
)

model.save(r'drive/My Drive/face_mask_detection.h5')

Epoch 1/20
76/76 [==============================] - 116s 2s/step - loss: 0.5121 - accuracy: 0.7694 - val_loss: 0.1630 - val_accuracy: 0.9756
Epoch 2/20
76/76 [==============================] - 117s 2s/step - loss: 0.2272 - accuracy: 0.9117 - val_loss: 0.0892 - val_accuracy: 0.9870
Epoch 3/20
76/76 [==============================] - 117s 2s/step - loss: 0.1651 - accuracy: 0.9361 - val_loss: 0.0756 - val_accuracy: 0.9805
Epoch 4/20
76/76 [==============================] - 115s 2s/step - loss: 0.1377 - accuracy: 0.9480 - val_loss: 0.0638 - val_accuracy: 0.9870
Epoch 5/20
76/76 [==============================] - 115s 2s/step - loss: 0.1226 - accuracy: 0.9526 - val_loss: 0.0601 - val_accuracy: 0.9837
Epoch 6/20
76/76 [==============================] - 115s 2s/step - loss: 0.0995 - accuracy: 0.9641 - val_loss: 0.0529 - val_accuracy: 0.9853
Epoch 7/20
76/76 [==============================] - 114s 2s/step - loss: 0.0899 - accuracy: 0.9670 - val_loss: 0.0474 - val_accuracy: 0.9886
Epoch 8/20
76

In [ ]:
predict = model.predict(testX,batch_size=Batch_size)
predict = np.argmax(predict,axis=1)
print(classification_report(testY.argmax(axis=1),predict,target_names=lb.classes_))


              precision    recall  f1-score   support

   with_mask       0.99      0.97      0.98       230
without_mask       0.98      0.99      0.99       384

    accuracy                           0.99       614
   macro avg       0.99      0.98      0.99       614
weighted avg       0.99      0.99      0.99       614

